In [2]:
import numpy as np
from random import randint
import math
import unittest
import operator
import numbers
import sys
import random

In [11]:
class TicTacToe:

    # There are 5812 legal board states that can be reached before there is a winner
    # http://brianshourd.com/posts/2012-11-06-tilt-number-of-tic-tac-toe-boards.html
    
    __asStr = True
    __bad_move_game_is_over = -1
    __bad_move_action_already_played = -2
    __bad_move_no_consecutive_plays = -3
    __good_move = 0 # value of a free action space on board
    __play = 0.01 # reward for playing an action
    __draw = 0.1 # reward for playing to end but no one wins
    __win = 0.05 # reward for winning a game
    __loss = -0.1 # reward (penalty) for losing a game
    __no_player = -2 # id of a non existent player i.e. used to record id of player that has not played
    __win_mask = np.full((1, 3),3,np.int8)
    actions =  {1:(0,0), 2:(0,1), 3:(0,2), 4:(1,0), 5:(1,1), 6:(1,2), 7:(2,0), 8:(2,1), 9:(2,2)}
    __num_actions = 9
    player_X = 1
    player_O = -1

    #
    # Return game to intial state, where no one has played
    # and the board contains no moves.
    #
    def reset(self):
        self.__board = np.zeros((3, 3),np.int8)
        self.__last_board = np.zeros((3, 3),np.int8)
        self.__game_over = False
        self.__game_drawn = False
        self.__player = TicTacToe.__no_player
        self.__last_player = TicTacToe.__no_player
        #NOT __Q = {} - as learning spans games, use the forget function !
        
    #
    # Constructor has no arguments as it just sets the game
    # to an intial up-played set-up
    #
    def __init__(self):
        self.__board = np.zeros((3, 3),np.int8)
        self.__last_board = np.zeros((3, 3),np.int8)
        self.__game_over = False
        self.__game_drawn = False
        self.__player = TicTacToe.__no_player
        self.__last_player = TicTacToe.__no_player
        self.__Q = {}
    
    #
    # return player as string "X" or "O"
    #
    def __player_to_str(self,player):
        if(player == TicTacToe.player_X): return "X"
        if(player == TicTacToe.player_O): return "O"
        return "?"
        
    #
    # Return a displayable version of the entire game.
    #
    def __str__(self):
        s = ""
        s += "Game Over: " + str(self.__game_over) +"\n"
        s += "Player :" + self.__player_to_str(self.__player) + "\n"
        s += "Current Board : \n" + str(self.__board)+ "\n"
        s += "Prev Player :" + self.__player_to_str(self.__last_player) + "\n"
        s += "Prev Current Board : \n" + str(self.__last_board)+ "\n"
        s += "State" + str(self.state()) + "\n"
        return s
    
    #
    # The learned Q Values for a given state if they exist
    #
    def Q_Vals_for_state(self,state):
        if(state in self.__Q):
            return(self.__Q[state])
        else:
            return(None)

    #
    # Return the list of valid actions
    #
    def list_actions(self):
        return list(self.actions)
    
    #
    # Is the game over ?
    #
    def game_over(self):
        return (self.__game_won() or not self.moves_left_to_take())
    
    #
    # Assume the move has been validated by move method
    # Make a copy of board before move is made and the last player
    #
    def __make_move(self, action, player):
        self.__last_board = np.copy(self.__board)
        self.__last_player = self.__player
        self.__player = player
        self.__board[TicTacToe.actions[action]] = player
        return
    
    #
    # Has a player already moved using the given action.
    #
    def __valid_move(self,action):
        return self.__board[TicTacToe.actions[action]] != self.__good_move
    
    #
    # If the proposed action is a valid move and the game is not
    # over. Make the given move (action) on behalf of the given 
    # player and update the game status.
    #
    # return the rawards (Player who took move, Observer)
    #
    def move(self, action, player):
        if(self.__game_won()) : return TicTacToe.__bad_move_game_is_over
        if(self.__valid_move(action)): return TicTacToe.__bad_move_action_already_played 
        if(player == self.__player): return TicTacToe.__bad_move_no_consecutive_plays 
        
        self.__make_move(action,player)

        if(self.__game_won()):
            self.__game_over = True
            self.__game_drawn = False
            return np.array([TicTacToe.__win,TicTacToe.__loss])
            
        if(not self.moves_left_to_take()):
            self.__game_over = True
            self.__game_drawn = True
            return np.array([TicTacToe.__draw,TicTacToe.__draw])

        return np.array([TicTacToe.__play,TicTacToe.__play])

    #
    # Return (flattened) Game Ended, Last Player, Last Board, Player, Board
    #
    def detailed_state(self):
        flattened_state = []
        if(self.__game_over):
            flattened_state.append(1)
        else:
            flattened_state.append(0)
        flattened_state.append(self.__last_player)
        flattened_state.append(self.__player)
        for itm in np.reshape(self.__last_board,9).tolist() : flattened_state.append(itm)
        for itm in np.reshape(self.__board,9).tolist() : flattened_state.append(itm)
            
        return flattened_state

    #
    # Return state of current board as simple vector or string
    #
    def state(self,tostr=False,plyr=None):
        flattened_state = []
        if(plyr == None):
            flattened_state.append(self.__player)
        else:
            flattened_state.append(plyr)
        
        for itm in np.reshape(self.__board,9).tolist() : flattened_state.append(itm)            
        if not tostr:
            return flattened_state
        else:
            return ''.join(str(e) for e in flattened_state)

    #
    # Show return the current board contents
    #
    def board(self):
        return self.__board
     
    #
    # Any row, column or diagonal with all player X or player O
    #
    def __game_won(self):
        rows = np.abs(np.sum(self.__board,axis=1))
        cols = np.abs(np.sum(self.__board,axis=0))
        diagLR = np.abs(np.sum(self.__board.diagonal()))
        diagRL = np.abs(np.sum(np.rot90(self.__board).diagonal()))
        
        if(np.sum(rows == self.__win_mask) > 0):
            return True
        if(np.sum(cols == self.__win_mask) > 0):
            return True
        if((np.mod(diagLR,3)) == 0) and diagLR > 0:
            return True
        if((np.mod(diagRL,3)) == 0) and diagRL > 0:
            return True
        return False

    #
    # Forget learning
    #
    def forget(self):
        self.__Q = {}

    #
    # Return which player goes next given the current player
    #
    def next_player(self,current_player):
        if(current_player == TicTacToe.player_O):
            return  TicTacToe.player_X
        else:
            return  TicTacToe.player_O

    #
    # Are there any remaining moves to be taken >
    #
    def moves_left_to_take(self):
        return (self.__board[np.where(self.__board == 0)]).size > 0
    
    #
    # Return a random action (move) that is still left
    # to make
    #
    def random_move(self):
        valid_moves = []
        random_action = None
        for key in self.list_actions():
            if(self.__board[TicTacToe.actions[key]] == self.__good_move):
                valid_moves.append(key)
         
        num_poss_moves = len(valid_moves)
        if(num_poss_moves > 0):
            random_action = valid_moves[randint(0, num_poss_moves-1)]
            return random_action
        else:
            return None
        
    @staticmethod
    def __action_to_index(action):
        return (int(action)-1)

    #
    # What moves are valid given or board or if not
    # for the current game board.
    #
    def what_are_valid_moves(self, bd = None):
        if bd == None: bd = self.__board
        vm = np.zeros(len(TicTacToe.actions))
        best_action = None
        for actn,index in TicTacToe.actions.items():
            if(bd[index] == 0):
                vm[int(actn)-1] = True
            else:
                vm[int(actn)-1] = False
        return vm

    #
    # Return a informed action (move) that is still left
    # to make based on given Q values for actions from a 
    # given state
    #
    def informed_move(self):
        # What moves are possible at this stage
        valid_moves = self.what_are_valid_moves()
        
        # Are there any moves ? 
        if(np.sum(valid_moves*np.full(self.__num_actions,1)) == 0):
            return None
           
        # Is there info learned for this state ?
        if self.state(tostr=True) in self.__Q:
            informed_actions = self.__Q[self.state(tostr=True)]
            informed_actions *= valid_moves
            best_action = np.max(informed_actions)
            if(best_action > 0):
                informed_actions = np.arange(1,self.__num_actions+1,1)[np.where(informed_actions == best_action)]
                best_action = informed_actions[randint(0, informed_actions.size-1)]
            else:
                best_action = None

        # If we found a good action then return that 
        # else pick a random action
        if best_action == None:
            actions = valid_moves*np.arange(1,self.__num_actions+1,1)
            actions = actions[np.where(actions > 0)]
            best_action = actions[randint(0,actions.size-1)]

        return int(best_action)

    #
    # Play a game until completion using Q values as learnt so far
    # via the informed_move() call.
    #    
    def play(self):
        self.reset()
        plyr = (TicTacToe.player_X,TicTacToe.player_O)[randint(0,1)] # Random player to start
        print(plyr)
        mv = None
        while(not self.game_over()):
            print(self.board())
            st = self.state(tostr=True)
            print(self.Q_Vals_for_state(st))
            print(st)
            mv = self.informed_move()
            self.move(mv,plyr)
            plyr = self.next_player(plyr)
        
        print("-")
        print(self.board())
        print(self.__Q[self.state(tostr=True)])
        return
    
    #
    # Add states to Q Value dictionary if not present
    #
    def add_states_if_missing(self,s1,s2,sp1,sp2):
        if s1 not in self.__Q:
            self.__Q[s1] = np.zeros((self.__num_actions))
        if sp1 not in self.__Q:
            self.__Q[sp1] = np.zeros((self.__num_actions))
        if s2 not in self.__Q:
            self.__Q[s2] = np.zeros((self.__num_actions))
        if sp2 not in self.__Q:
            self.__Q[sp2] = np.zeros((self.__num_actions))

    #
    # Update the Q values for the given player state and
    # the given reward
    #
    def update_Q_Values_for_player(self,mv,s,sp,reward,learning_rate,discount_rate):
        (self.__Q[s])[mv-1] = learning_rate * (self.__Q[s])[mv-1] + (1-learning_rate) * (reward + discount_rate * np.max(self.__Q[sp]))
    
    #
    # Run simulation to estimate Q values for state, action pairs. Random exploration policy
    # which should be tractable with approx 6K valid board states.
    #
    def estimate_Q_values(self,num_simulations):
        exploration = 1.0
        decay = (1.0/num_simulations)
        learning_rate0 = 0.05
        learning_rate_decay = 0.1
        discount_rate = 0.95
        reward = 0
        s = None
        sp = None
        sim = 0 
        score = {TicTacToe.__draw:0,TicTacToe.__win:0}
        while(sim < num_simulations):
            self.reset()
            plyr = (TicTacToe.player_X,TicTacToe.player_O)[randint(0,1)] # Random player to start
            nxt_plyr = self.next_player(plyr)
            mv = None
            while(not self.game_over()):
                s1 = self.state(self.__asStr,plyr)
                s2 = self.state(self.__asStr,nxt_plyr)
                
                if(True):#random.random() < (exploration-(decay*sim))):
                    mv = self.random_move()
                else:
                    mv = self.informed_move()
                reward = self.move(mv,plyr)
                sp1 = self.state(self.__asStr,plyr)
                sp2 = self.state(self.__asStr,nxt_plyr)
                
                learning_rate = learning_rate0 / (1 + (sim * learning_rate_decay))
                
                self.add_states_if_missing(s1,s2,sp1,sp2)
                
                self.update_Q_Values_for_player(mv,s1,sp1,reward[0],learning_rate,discount_rate)
                self.update_Q_Values_for_player(mv,s2,sp2,reward[1],learning_rate,discount_rate)
                plyr = nxt_plyr
                nxt_plyr = self.next_player(plyr)

            sim += 1
            score[reward[0]] += 1
            if (sim % 1000) == 0 : 
                print(str(sim)+" Win : "+str(round((score[TicTacToe.__win]/sim)*100,0))+"% Draw: " + str(round((score[TicTacToe.__draw]/sim)*100,0))+"%")
        print(score)
        return self.__Q
        

In [12]:
random.seed(42)
np.random.seed(42)
game = TicTacToe()
QV = game.estimate_Q_values(50000)

1000 Win : 86.0% Draw: 14.0%
2000 Win : 88.0% Draw: 12.0%
3000 Win : 88.0% Draw: 12.0%
4000 Win : 88.0% Draw: 12.0%
5000 Win : 88.0% Draw: 12.0%
6000 Win : 87.0% Draw: 13.0%
7000 Win : 87.0% Draw: 13.0%
8000 Win : 87.0% Draw: 13.0%
9000 Win : 87.0% Draw: 13.0%
10000 Win : 87.0% Draw: 13.0%
11000 Win : 87.0% Draw: 13.0%
12000 Win : 87.0% Draw: 13.0%
13000 Win : 87.0% Draw: 13.0%
14000 Win : 87.0% Draw: 13.0%
15000 Win : 87.0% Draw: 13.0%
16000 Win : 87.0% Draw: 13.0%
17000 Win : 87.0% Draw: 13.0%
18000 Win : 87.0% Draw: 13.0%
19000 Win : 87.0% Draw: 13.0%
20000 Win : 87.0% Draw: 13.0%
21000 Win : 87.0% Draw: 13.0%
22000 Win : 87.0% Draw: 13.0%
23000 Win : 87.0% Draw: 13.0%
24000 Win : 87.0% Draw: 13.0%
25000 Win : 87.0% Draw: 13.0%
26000 Win : 87.0% Draw: 13.0%
27000 Win : 87.0% Draw: 13.0%
28000 Win : 87.0% Draw: 13.0%
29000 Win : 87.0% Draw: 13.0%
30000 Win : 87.0% Draw: 13.0%
31000 Win : 87.0% Draw: 13.0%
32000 Win : 87.0% Draw: 13.0%
33000 Win : 87.0% Draw: 13.0%
34000 Win : 87.0% D

In [28]:
def informed_move(game,st,rnd):
    # What moves are possible at this stage
    valid_moves = game.what_are_valid_moves()
        
    # Are there any moves ? 
    if(np.sum(valid_moves*np.full(9,1)) == 0):
        return None
    
    best_action = None
    if(not rnd):
        # Is there info learned for this state ?
        informed_actions = game.Q_Vals_for_state(st)
        if not informed_actions is None:
            print("**I**")
            informed_actions *= valid_moves
            best_action = np.max(informed_actions)
            if(best_action > 0):
                informed_actions = np.arange(1,9+1,1)[np.where(informed_actions == best_action)]
                best_action = informed_actions[randint(0, informed_actions.size-1)]
            else:
                best_action = None

    # If we found a good action then return that 
    # else pick a random action
    if best_action == None:
        print("**R**")
        actions = valid_moves*np.arange(1,9+1,1)
        actions = actions[np.where(actions > 0)]
        best_action = actions[randint(0,actions.size-1)]

    return int(best_action)

In [30]:
def play(game):
        game.reset()
        plyr = (TicTacToe.player_X,TicTacToe.player_O)[randint(0,1)] # Random player to start
        print(plyr)
        mv = None
        while(not game.game_over()):
            print("--")
            print(game.board())
            st = game.state(True,plyr)
            print(st)
            QV = game.Q_Vals_for_state(st)
            print(QV)
            mx = np.max(game.Q_Vals_for_state(st))
            print(mx)
            print(QV*(QV==mx))
            if(plyr == TicTacToe.player_X):
                mv = informed_move(game,st,False)
            else:
                mv = informed_move(game,st,True)            
            print("Move:" +str(mv))
            game.move(mv,plyr)
            print(game.board())
            plyr = game.next_player(plyr)
            print("--\n")
        print("-")
        print(game.board())
        print(game.Q_Vals_for_state(st))
        return

In [45]:
def _game_won(bd,plyr=None):
    
    if not plyr is None: bd = (bd==plyr)*1
        
    rows = np.abs(np.sum(bd,axis=1))
    cols = np.abs(np.sum(bd,axis=0))
    diagLR = np.abs(np.sum(bd.diagonal()))
    diagRL = np.abs(np.sum(np.rot90(bd).diagonal()))        
    
    if(np.sum(rows == 3) > 0):
        return True
    if(np.sum(cols == 3) > 0):
        return True
    if((np.mod(diagLR,3)) == 0) and diagLR > 0:
        return True
    if((np.mod(diagRL,3)) == 0) and diagRL > 0:
        return True
    return False

In [ ]:
def _play(game):
        game.reset()
        plyr = (TicTacToe.player_X,TicTacToe.player_O)[randint(0,1)] # Random player to start
        mv = None
        while(not game.game_over()):
            st = game.state(True,plyr)
            QV = game.Q_Vals_for_state(st)
            mx = np.max(game.Q_Vals_for_state(st))
            if(plyr == TicTacToe.player_X):
                mv = informed_move(game,st,False)
            else:
                mv = informed_move(game,st,True)            
            game.move(mv,plyr)
            plyr = game.next_player(plyr)
        return game.board()

In [47]:
print(game.board())
print(_game_won(game.board()))
print(_game_won(game.board(),1))
print(_game_won(game.board(),-1))

[[ 1  1 -1]
 [ 1 -1  0]
 [-1 -1  1]]
True
False
True


In [31]:
play(game)

1
--
[[0 0 0]
 [0 0 0]
 [0 0 0]]
1000000000
[ 0.13365796  0.13365796  0.13365796  0.13365796  0.13365796  0.13365796
  0.13365796  0.13365796  0.13365796]
0.133657956871
[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.13365796]
**I**
Move:9
[[0 0 0]
 [0 0 0]
 [0 0 1]]
--

--
[[0 0 0]
 [0 0 0]
 [0 0 1]]
-1000000001
[ 0.13016627  0.13016627  0.13016627  0.13016627  0.13016627  0.13016627
  0.13016627  0.13016627  0.        ]
0.130166270391
[ 0.          0.          0.          0.          0.13016627  0.          0.
  0.          0.        ]
**R**
Move:8
[[ 0  0  0]
 [ 0  0  0]
 [ 0 -1  1]]
--

--
[[ 0  0  0]
 [ 0  0  0]
 [ 0 -1  1]]
10000000-11
[ 0.12649081  0.12649081  0.12649081  0.12649081  0.12649081  0.12649081
  0.12649081  0.          0.        ]
0.126490810937
[ 0.          0.          0.          0.12649081  0.          0.          0.
  0.          0.        ]
**I**
Move:4
[[ 0  0  0]
 [ 1  0  0]
 [ 0 -1  1]]
--

--
[[ 0  0  0]
 [ 1  

In [ ]:
informed_actions = game.Q_Vals_for_state("-1-10-1010110")
#informed_actions *= valid_moves
best_action = np.max(informed_actions)
if(best_action > 0):
    informed_actions = np.arange(1,10,1)[np.where(informed_actions == best_action)]
    best_action = informed_actions[randint(0, informed_actions.size-1)]
else:
    best_action = None
print(best_action)

In [ ]:
#@staticmethod
def state_to_board(st): 
    if(st[0]=='-'):
        st = st[2:] # ignore first number as this is the player not a board element
    else:
        st = st[1:]
    bd = np.zeros((3, 3),np.int8)
    i = 0
    j = 0
    plyr = 1
    for c in st:
        if c == '-':
            plyr = -1
        else:
            bd[i,j] = int(c) * plyr
            if plyr == -1: plyr = 1
            j+=1
            if j == 3:
                j=0
                i+=1
    return bd

def valid_moves(board):
    vm = np.zeros(len(TicTacToe.actions))
    best_action = None
    for actn,index in TicTacToe.actions.items():
        if(bd[index] == 0):
            vm[int(actn)-1] = True
        else:
            vm[int(actn)-1] = False
    return vm

In [ ]:
#alla = {1:(0,0), 2:(0,1), 3:(0,2), 4:(1,0), 5:(1,1), 6:(1,2), 7:(2,0), 8:(2,1), 9:(2,2)}
print(type(TicTacToe.actions))
bd = state_to_board("-1-10-1010110")
vm = valid_moves(bd)
print(bd)
print(vm)

In [ ]:
print(valid_moves(state_to_board("-1-10-1010110")))

In [ ]:
#
# Run A Single Test
#
test_to_run = "test_one"
suite = unittest.TestSuite()
suite.addTest(testTicTacToe(test_to_run))
runner = unittest.TextTestRunner()
runner.run(suite)

In [ ]:
#
# Run All Tests.
#
tests = testTicTacToe()
suite = unittest.TestLoader().loadTestsFromModule(tests)
unittest.TextTestRunner().run(suite)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [ ]:
class playTicTacToe:
    __actor_network_name = "net/actor"
    __critic_network_name = "net/critic"
    __x_units = 21

    def __init__(self):
        return
    
    #
    # Build a four layer fully connected Network.
    # 21 -> relu -> 21 -> relu -> 21 -> relu -> 21 -> relu
    #
    def construct_network(X_state,network_name):
        return